In [2]:
import numpy as np

### Open Serial Port
You might need to change 'COM'

In [3]:
import serial as ser
s = ser.Serial ('COM5', baudrate = 9600, parity = 'N', bytesize = 8, stopbits = 1, timeout = None)

ModuleNotFoundError: No module named 'serial'

### Define Rotation / Transformation Functions

In [4]:
def Rx (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [1, 0,  0, 0],
        [0, c, -s, 0],
        [0, s,  c, 0],
        [0, 0,  0, 1]
    ])

def Ry (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, 0,  s, 0],
        [0, 1,  0, 0],
        [-s, 0, c, 0],
        [0, 0,  0, 1]
    ])

def Rz (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, -s,  0, 0],
        [s, c,  0, 0],
        [0, 0, 1, 0],
        [0, 0,  0, 1]
    ])

def Tx (x):
    return np.array ([
        [1, 0, 0, x],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Ty (y):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, y],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Tz (z):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

In [10]:
def move(a):
    
    #custom start point
    #startpoint = np.array([90,85,175,180,90,65])
    startpoint= np.array([90, 90, 87, 90, 90, 65])
    a_calc = startpoint + a
    #print(a)
    command = f'{a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)
    
    if res[2,3] < 0:
       raise Exception(f'Illegal z value: {res[2,3]} lower than 0')
    else:
        s.write(command.encode())
        return res
    


In [20]:
def move_nocheck(a):
    
    #custom start point
    #startpoint = np.array([90,85,175,180,90,65])
    startpoint= np.array([90, 90, 87, 90, 90, 65])
    a_calc = startpoint + a
    #print(a)
    command = f'{a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)

    s.write(command.encode())
    return res
    


In [15]:
Rz(-90)@Tz(40)

#1. Rot z Base 
#2. Tz 71.5mm
#3  Rot y Shoulder
#4  Tz 125mm
#5  Rot y Elbow
#6  Tz 125mm
#7  Rot y Wrist
#8  Tz 60mm
#9 (Wrist Rot)
#10 Tz 132mm
#11 Tx OFFSET 12.5mm for new start position

#def transformation(a):
#    return Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)

In [15]:

a = np.array([0,0,0,0,0,0])

move(a)
#transformation(a)

array([[   1. ,    0. ,    0. , -125. ],
       [   0. ,    1. ,    0. ,    0. ],
       [   0. ,    0. ,    1. ,  513.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [25]:
### Preset Demo

#move(np.array([0,-20,-90,-90,0,0]))
move_nocheck(np.array([0,-90,-90,-30,0,0]))

array([[  -0.8660254 ,    0.        ,    0.5       , -154.        ],
       [   0.        ,    1.        ,    0.        ,    0.        ],
       [  -0.5       ,    0.        ,   -0.8660254 , -219.77687753],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [5]:
def rad(phi):
    return phi * np.pi / 180
def c(phi):
    return np.cos(phi)
def s(phi):
    return np.sin(phi)

def dh_calc(a,alpha,d,theta):
    return np.array([
        [c(theta), -s(theta), 0, a],
        [s(theta)*c(alpha), c(theta)*c(alpha), -s(alpha), -d * s(alpha)],
        [s(theta)*s(alpha), c(theta)*s(alpha), c(alpha), d * c(alpha)],
        [0,0,0,1]
    ])

In [10]:
b_a = rad(-90)
r1 = np.array([
    [c(b_a),-s(b_a),0,0],
    [s(b_a),c(b_a),0,0],
    [0,0,1,0],
    [0,0,0,1]
])

t1 = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,40],
    [0,0,0,1]
])



array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

### Warning: Experimental Code for DH
Never tested and probably not working

In [15]:
#warning: rad
#Ansatz DH, nicht überprüft

a1 = rad(90)
a2 = rad(90)
a3 = rad(180)
a4 = rad(0)
a5 = rad(90)


dh_parameters = np.array ([
    [0, rad(90), 71.5, a1],
    [125, 0, 0, a2+rad(90)],
    [125, 0, 0, a3],
    [0, rad(90), 0, a4 - rad(90)],
    [0, 0, 192, a5]
])

dh_matrices = []

for row in dh_parameters:
    dh_matrices.append(dh_calc(row[0],row[1],row[2],row[3]))
    


dh_matrices[0]@dh_matrices[1]@dh_matrices[2]@dh_matrices[3]@dh_matrices[4]

array([[ 3.06161700e-16, -6.12323400e-17,  1.00000000e+00,
         1.92000000e+02],
       [ 6.12323400e-17, -1.00000000e+00, -6.12323400e-17,
        -7.15000000e+01],
       [ 1.00000000e+00,  6.12323400e-17, -3.06161700e-16,
        -5.87830464e-14],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

## Reverse Kinematics

In [88]:
#target = np.array([x,y,z])
target = np.array([240,50,10])

h = 1E-8

def f(a):
    
    #Implement three axis of freedom
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(0) @ Tz(60) @ Tz(132)
    
    delta = target.reshape(3,1) - res[0:3,3:4]
    #print(f'i calc {delta} = {target.reshape(3,1)} - {res[0:3,3:4]}')
    return delta

def f_d(x0):
    y0 = f (x0)
    #print (y0)
    x1 = np.copy (x0); x1 [0] += h; y1 = f (x1); df1 = (y1 - y0) / h;
    x2 = np.copy (x0); x2 [1] += h; y2 = f (x2); df2 = (y2 - y0) / h;
    x3 = np.copy (x0); x3 [2] += h; y3 = f (x3); df3 = (y3 - y0) / h;
    J = np.concatenate ((df1, df2, df3), axis = 1)
    dx = np.linalg.solve (J, y0)
    print (dx)

def rev_k(x0):
    y0 = f(x0)
    
    col1 = np.copy(x0)
    col1[0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0)
    col2[1] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0)
    col3[2] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    J = np.concatenate((col1_df,col2_df,col3_df),axis=1)
    
    dx = np.linalg.solve (J,y0)
    print(dx)
    print(x0)


#Jacobimatrix
def j(x0):
    y0 = f(x0)
    
    col1 = np.copy(x0)
    col1[0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0)
    col2[1] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0)
    col3[2] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    return np.concatenate((col1_df,col2_df,col3_df),axis=1)

#Iterative newton-raphson approximation for angle correction
def approximate(x0):
    x = np.copy(x0).reshape(3,1)
    for i in range(15):
        #x -= np.linalg.solve(j(x.reshape(3,)),f(x.reshape(3,)))
        x -= np.linalg.inv(j(x.reshape(3,))) @ f(x.reshape(3,))
        print(f'n={i+1}\n{x}\n')

approximate(np.array([10,10,20]).astype(float))
#rev_k(np.array([20,10,20]).astype(float))

n=1
[[ -25.80769121]
 [-408.58166468]
 [ 661.87462883]]

n=2
[[-42.22998886]
 [-30.26262981]
 [156.74717758]]

n=3
[[19.9685796 ]
 [61.14826352]
 [83.65304351]]

n=4
[[-13.68712878]
 [ 18.41243903]
 [108.96734082]]

n=5
[[-6.25698597]
 [43.6980366 ]
 [76.90980908]]

n=6
[[-7.83960167]
 [45.09425652]
 [73.07040447]]

n=7
[[-7.80014401]
 [45.29358472]
 [72.83182863]]

n=8
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

n=9
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

n=10
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

n=11
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

n=12
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

n=13
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

n=14
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

n=15
[[-7.80018788]
 [45.29376373]
 [72.83149091]]

